In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D,SeparableConv2D,BatchNormalization,AveragePooling2D,Activation,Dense
from tensorflow.keras import Input
import tensorflow_datasets as tfds

In [ ]:
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.29.16.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.29.16.82:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
(x_train, y_train), (x_test, y_test) = tfds.as_numpy(tfds.load(
    'Cifar10',
    split=['train', 'test'],
    batch_size=-1,
    as_supervised=True,
))

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteAZXWEX/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteAZXWEX/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [ ]:
x_train.shape,y_train.shape

((50000, 32, 32, 3), (50000,))

In [ ]:
x_test.shape,y_test.shape

((10000, 32, 32, 3), (10000,))

In [ ]:
y_train = tf.one_hot(y_train,depth=10)
y_test = tf.one_hot(y_test,depth=10)

In [ ]:
y_test.shape, y_train.shape

(TensorShape([10000, 10]), TensorShape([50000, 10]))

In [ ]:
def mobilenetv1(x,alph = 1): # 224 224 3
    def dw(x,dw_pad,conv_f,conv_st):
            x = DepthwiseConv2D(kernel_size=(3,3),padding = 'same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x) 
            x = Conv2D(filters= conv_f,kernel_size=(1,1),strides=conv_st,padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            return x
    x = Conv2D(filters=int(32*alph),kernel_size=(3,3),strides=2,padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = dw(x,'same',int(64*alph),1)
    x = dw(x,'valid',int(128*alph),2)
    x = dw(x,'same',int(128*alph),1)
    x = dw(x,'same',int(256*alph),2)
    x = dw(x,'same',int(256*alph),1)
    x = dw(x,'valid',int(512*alph),2)
    for i in range(5):
        x = dw(x,'same',int(512*alph),1) 
    x = dw(x,'valid',int(1024*alph),2)
    x = dw(x,'same',int(1024*alph),1)
    return x


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping

filename = 'checkpoint-epoch-{}-batch-{}-trial-001.h5'.format(30, 128)
checkpoint = ModelCheckpoint(filename,monitor='val_loss', verbose=1,save_best_only=True, mode='auto')
earlystopping = EarlyStopping(monitor='val_loss',patience=7)
reduceLR = ReduceLROnPlateau( monitor='val_loss',factor=0.5,patience=3,)


with strategy.scope():
    inputs = Input(shape = (32,32,3),dtype=np.float32)
    x = mobilenetv1(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = Dense(10, activation='softmax')(x)
    model = tf.keras.models.Model(inputs,outputs)
    nadam = tf.keras.optimizers.Nadam(lr=0.01)
    model.compile(optimizer=nadam,loss='categorical_crossentropy',metrics=['accuracy'])


model.fit(x_train,y_train,batch_size=128, epochs=30,validation_split=0.1,callbacks=[reduceLR,checkpoint,earlystopping])

Epoch 1/30
352/352 [==============================] - 52s 57ms/step - loss: 2.6509 - accuracy: 0.2099 - val_loss: 2.3848 - val_accuracy: 0.1094

Epoch 00001: val_loss improved from inf to 2.38479, saving model to checkpoint-epoch-30-batch-128-trial-001.h5
Epoch 2/30
352/352 [==============================] - 9s 26ms/step - loss: 1.7385 - accuracy: 0.3575 - val_loss: 2.6759 - val_accuracy: 0.3192

Epoch 00002: val_loss did not improve from 2.38479
Epoch 3/30
352/352 [==============================] - 9s 25ms/step - loss: 1.5415 - accuracy: 0.4381 - val_loss: 13.7431 - val_accuracy: 0.1794

Epoch 00003: val_loss did not improve from 2.38479
Epoch 4/30
352/352 [==============================] - 9s 26ms/step - loss: 1.4609 - accuracy: 0.4781 - val_loss: 41.5583 - val_accuracy: 0.1700

Epoch 00004: val_loss did not improve from 2.38479
Epoch 5/30
352/352 [==============================] - 9s 26ms/step - loss: 1.2826 - accuracy: 0.5419 - val_loss: 1.2438 - val_accuracy: 0.5618

Epoch 00005: 

In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 7s 17ms/step - loss: 5.5110 - accuracy: 0.6352


[5.511025905609131, 0.6351999640464783]

In [ ]:
model.save("mobilenetv1.h5")